In [1]:
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [ ]:
!jt -t 

In [2]:
# -*- coding: utf-8 -*- 
from Bio import PDB
import networkx as nx
import numpy as np
import os
import scipy.sparse as sp
import pickle
import torch
import dgl
import pandas as pd
import matplotlib.pyplot as plt
import mdtraj as md
import time
import itertools

Using backend: pytorch


In [2]:
aa_prop = pd.DataFrame(map(lambda x: x.split(","), "A,1.29,0.9,0,0.049,1.8,0,0,0.047,0.065,0.78,67,1,0,0,1,1;\
                           C,1.11,0.74,0,0.02,2.5,-2,0,0.015,0.015,0.8,86,1,1,-1,0,1;\
                           D,1.04,0.72,-1,0.051,-3.5,-2,1,0.071,0.074,1.41,91,1,0,1,1;\
                           E,1.44,0.75,-1,0.051,-3.5,-2,1,0.094,0.089,1,109,1,0,1,0,1;\
                           F,1.07,1.32,0,0.051,2.8,0,0,0.021,0.029,0.58,135,1,1,-1,0,1;\
                           G,0.56,0.92,0,0.06,-0.4,0,0,0.071,0.07,1.64,48,1,0,1,1,1;\
                           H,1.22,1.08,0,0.034,-3.2,1,1,0.022,0.025,0.69,118,1,0,-1,0,1;\
                           I,0.97,1.45,0,0.047,4.5,0,0,0.032,0.035,0.51,124,1,1,-1,0,1;\
                           K,1.23,0.77,1,0.05,-3.9,2,1,0.105,0.08,0.96,135,1,0,1,0,1;\
                           L,1.3,1.02,0,0.078,3.8,0,0,0.052,0.063,0.59,124,1,1,-1,1,1;\
                           M,1.47,0.97,0,0.027,1.9,0,0,0.017,0.016,0.39,124,1,1,1,0,1;\
                           N,0.9,0.76,0,0.058,-3.5,0,1,0.062,0.053,1.28,96,1,0,1,1,1;\
                           P,0.52,0.64,0,0.051,-1.6,0,0,0.052,0.054,1.91,90,1,0,1,0,1;\
                           Q,1.27,0.8,0,0.051,-3.5,1,1,0.053,0.051,0.97,114,1,0,1,0,1;\
                           R,0.96,0.99,1,0.066,-4.5,2,1,0.068,0.059,0.88,148,1,0,1,1,1;\
                           S,0.82,0.95,0,0.057,-0.8,-1,1,0.072,0.071,1.33,73,1,0,1,1,1;\
                           T,0.82,1.21,0,0.064,-0.7,-1,0,0.064,0.065,1.03,93,1,0,0,1,1;\
                           V,0.91,1.49,0,0.049,4.2,0,0,0.048,0.048,0.47,105,1,1,-1,0,1;\
                           W,0.99,1.14,0,0.022,-0.9,1,1,0.007,0.012,0.75,163,1,1,-1,0,1;\
                           Y,0.72,1.25,0,0.07,-1.3,-1,1,0.032,0.033,1.05,141,1,1,-1,1,1".split(";")), columns=['aminoacid', 'alpha', 'beta', 'charge', 'core', 'hydropathy', 'pH', 'polarity', 'rim', 'surface', 'turn', 'volume', 'count', 'strength', 'disorder', 'high_contact', 'count'], index=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])
aa_prop = aa_prop.drop(columns=["count"])
#print(aa_prop)

amino_acid = ['Gly','Ala','Val','Leu','Ile','Phe','Trp','Tyr','Asp','Asn','Glu','Lys','Gln',
              'Met','Ser','Thr','Cys','Pro','His','Arg']
acid_low = ['G','A','V','L','I','F','W','Y','D','N','E','K','Q','M','S','T','C','P','H','R']

aa_dic = {}
for i in range(20):
    aa_dic[amino_acid[i].upper()] = acid_low[i]
print(aa_dic)

{'GLY': 'G', 'ALA': 'A', 'VAL': 'V', 'LEU': 'L', 'ILE': 'I', 'PHE': 'F', 'TRP': 'W', 'TYR': 'Y', 'ASP': 'D', 'ASN': 'N', 'GLU': 'E', 'LYS': 'K', 'GLN': 'Q', 'MET': 'M', 'SER': 'S', 'THR': 'T', 'CYS': 'C', 'PRO': 'P', 'HIS': 'H', 'ARG': 'R'}


In [3]:
from sklearn.preprocessing import label_binarize
index=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
acid_dict={}
for i in range(len(index)):
    acid_dict[index[i]]=label_binarize([i], classes=range(0,20))[0]

In [4]:
data=acid_dict
df=pd.DataFrame(data,index=index)

In [9]:
df

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
A,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
I,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
K,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
L,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [5]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [6]:
path = '/home/dldx/data_sda/gp120_tied/trajs_5fyj/'
dirs = os.listdir(path)
dirs

['3', '1', '.ipynb_checkpoints', '2']

In [7]:
def res_distance(residues_top,res_len):
    # residues_distance_unit = 'nanometers'
    res_index=range(res_len)
    residues_distance = md.compute_distances(residues_top,itertools.combinations(res_index,2))
    n=np.zeros((res_len,res_len))
    count=0
    #print(residues_distance)
    
    for i in range(0,res_len):
        for j in range(i+1,res_len):
        
            n[i][j]=residues_distance[0][count]
            n[j][i]=n[i][j]
            count+=1
    return n
def add_dis(n):
    res=[]
    rnge=len(n)
    for i in range(rnge):
        d={}
        for j in range(rnge):
            d[j]=n[i][j]
        
        s3 = sorted(d.items(), key=lambda d:d[1], reverse=False)
        for k in range(rnge):
            if s3[k][1]>0.8: #0.8nm=8Å
                flag=k
                break
        s3=s3[:20]
        res.append(s3)   
    return(res)

def cos_angle(xyz1,xyz2):
    angle1 = np.dot(xyz1 / 10, xyz2 / 10) / (np.linalg.norm(xyz1 / 10) * (np.linalg.norm(xyz2 / 10)))
    return angle1

In [11]:
graph = []
for index, name in enumerate(dirs):
    if name not in ['3', '1', '2']:
        continue
    tem_path = os.path.join(path, name)
    pdb = os.listdir(tem_path)
    for pdb_ in pdb:

        #print(name)
        end_path = os.path.join(path, name)
        pdb_path = os.path.join(end_path, pdb_,pdb_ + '_5fyk.pdb')
        print(pdb_path)
        xtc_path=os.path.join(end_path, pdb_,'pro.xtc')
        
        traj = md.load(xtc_path,top=pdb_path)
        #print(traj)
        
        for i in range(1):
#             if i%50!=0:
#                 continue
            print(i)
            residues_top=traj[i]
            residues_index = residues_top.topology.select('name CA')
            #print(residues_index)
            res_len=len(residues_index)
        
            feat_num=20#结点特征维度
            feat = np.zeros((res_len, feat_num))
        
            G = dgl.DGLGraph()
            G.add_nodes(res_len)
        
            #计算节点距离
            n=res_distance(residues_top,res_len)
            res=add_dis(n)
            #print(res)
            edge_w = []
            angle_w = []

            for index1 in range (res_len):
            
                e= res[index1]
                xyz1=residues_top.xyz[0, residues_index[index1],:]
                #print(e)
                for t in e:
                    G.add_edge(index1, t[0])
                    #print(type(t[1]))='torch.float64'
                    edge_w.append(t[1])
                
                
                    xyz2=residues_top.xyz[0, residues_index[t[0]],:]    
                    #print(xyz1,xyz2)
                    angle1=cos_angle(xyz1,xyz2)
                
                    angle_w.append(angle1)
            

                resname=str(residues_top.topology.residue(3))
                feat[index1][0:20] = np.array(df.loc[aa_dic[resname[:3]]].values[0:],
                                    dtype=np.float64)

            edge_w = torch.tensor(edge_w).unsqueeze(1)  # 9360
        
            #print(edge_w.dtype)
            angle_w = torch.tensor(angle_w).unsqueeze(1)
            angle_w=angle_w.double()
            #print(angle_w.dtype)
        
            #边的特征拼接
            edge_w = torch.cat([edge_w, angle_w], dim=1)
            G.edata["w"] = edge_w      
        
            feature = normalize(sp.csr_matrix(feat))
            feature = torch.tensor(np.array(feature.todense()))
            G.ndata["feat"] = feature
            graph.append((G, int(name)-1,name+'-'+pdb_+'-'+str(i)))
dataset_name = "/home/dldx/UniRep/dataset/data_45_onehot.p"
with open(dataset_name, 'wb') as f:
    pickle.dump(graph, f)

/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/B2YFU4/B2YFU4_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/A4ZPW7/A4ZPW7_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/Q5G5U6/Q5G5U6_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/B2YFV4/B2YFV4_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/B2YFS0/B2YFS0_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/A0MTL0/A0MTL0_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/A1EAG8/A1EAG8_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/B2YFP6/B2YFP6_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/A1EAH3/A1EAH3_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/A4ZPW9/A4ZPW9_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/A4ZPX1/A4ZPX1_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/Q5G5V2/Q5G5V2_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/B2YFP1/B2YFP1_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/trajs_5fyj/3/A1EAH6/A1EAH6_5fyk.pdb
0
/home/dldx/data_sda/gp120_tied/tra

In [15]:
feature[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0.], dtype=torch.float64)

In [6]:
# A4ZPX2
tied1_name_l = ['Q5G5V8','Q5G5V4','Q6EG58','A4ZPW4','P04578','Q4QXE8','A8CVW4','Q2MKA8','A1EAH8','A1EAI4','Q27Q69','Q70014','B2YFU9','B2YFQ5','O89960']
# A1EAI3
tied2_name_l = ['Q5G5U7','Q5G5U5','A7KVY7','Q5G5V1','B0FBI6','Q5G5V5','A1EAI2','A1EAI0','Q6TCV7','Q202K1','Q202K7','Q27Q74','A1EAI7','B2YFT0','Q8JDI3']
# A4ZPW9
tied3_name_l = ['A4ZPX1','A4ZPW7','A4ZPW8','Q5G5V2','Q5G5U6','A1EAG8','A1EAH7','A0MTL0','A1EAH6','A1EAH3','B2YFP6','B2YFP1','B2YFU4','B2YFV4','B2YFS0']
tied_name=tied1_name_l+tied2_name_l+tied3_name_l

In [7]:
len(tied_name)

45

In [8]:
new_graph=[]
for g in graph:
    name=g[-1].split('-')[1]
    if name in tied_name:
        new_graph.append(g)
dataset_name = "/home/dldx/UniRep/dataset/data_45_onehot.p"
with open(dataset_name, 'wb') as f:
    pickle.dump(new_graph, f)

In [9]:
dataset_name = "/home/dldx/UniRep/dataset/data_45_onehot.p"
with open(dataset_name, 'rb') as f:
    graph=pickle.load(f)

In [10]:
len(graph)

45